In [1]:
from data.processing import ParseAll

subset_name = "autotagging_top50tags"
subset_file = f"E:/mtg-jamendo-dataset/data/{subset_name}.tsv"
data_directory = "E:/mtg-jamendo/"
output_directory = f"D:/SongsDataset/melspec-mtg-jamendo"
subset_data = f'E:/mtg-jamendo-dataset/stats/{subset_name}/all.tsv'
ParseAll(subset_file, subset_data, data_directory, output_directory)

Reading: 54380 tracks, 11107 albums, 3517 artists
Processing tracks...


  4%|▎         | 2/54 [59:54<25:57:39, 1797.29s/it]


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 64 vs 0

In [1]:
from utils.Config import Config
from data.data_utils import *

directory = "melspec-dataset-top-50-LIBROSA-256-Triplet"
data_directory = "E:/mtg-jamendo/"
subset_file_name = "autotagging_top50tags"

model_name = "Myna-CLS"
prefetch_factor = 1
config = Config(
        save_path=f"trained_models\\{model_name}\\",
        num_epochs=512,
        learning_rate=3e-4,
        weight_decay=1e-5,
        num_workers=2,
        batch_size= 484,
        eval_batch_size=484,
        dtype=torch.float32
    )

In [2]:
from torch.utils.data import DataLoader

large_directory = directory

train_dataset = StreamViewDataset(f"E:\\SongsDataset\\raw_30s_melspecs", views=2)
test_dataset  = StreamViewDataset(f"E:\\SongsDataset\\raw_30s_melspecs", views=2)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    # num_workers=config.num_workers,
    # prefetch_factor=prefetch_factor,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    # num_workers=config.num_workers,
    # prefetch_factor=prefetch_factor,
)

In [ ]:
from utils import misc
from models.Myna import Myna
from training.contrastive_training import train_contrastive

model = Myna(
    image_size=(96, 256),
    channels=1,
    patch_size=(16, 16),
    latent_space=128,
    d_model=384,
    depth=12,
    heads=6,
    mlp_dim=1536,
    mask_ratio=0.9,
    use_cls=True,
    alibi=False
)

print(f"{misc.model_size(model)} Parameters")
train_contrastive(model, test_dataloader, train_dataloader, config, convex=False, start_epoch=0, views=2)

21425536 Parameters


  0%|          | 0/116 [00:00<?, ?it/s]

In [3]:
from training.contrastive_training import evaluate_contrastive

model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-ALIBI\\Epoch-103.pt", weights_only=False)
model.mask_ratio = 0.0
same_song_contrastive_loss = evaluate_contrastive(model, test_dataloader, Config, test_masked=False)
print(same_song_contrastive_loss)

  0%|          | 0/3 [00:00<?, ?it/s]

2.8604929447174072


In [4]:
from training.contrastive_training import evaluate_contrastive

model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-Sinusoidal\\Epoch-96.pt", weights_only=False)
model.mask_ratio = 0.0
same_song_contrastive_loss = evaluate_contrastive(model, test_dataloader, Config, test_masked=False)
print(same_song_contrastive_loss)

  0%|          | 0/3 [00:00<?, ?it/s]

2.715553879737854


In [7]:
from utils import misc
from models.Myna import Myna
from training.contrastive_training import train_contrastive

model = Myna(
    image_size=(128, 256),
    channels=1,
    patch_size=(16, 16),
    latent_space=128,
    d_model=384,
    depth=12,
    heads=6,
    mlp_dim=1536,
    mask_ratio=0.9,
    use_cls=True,
    alibi=False
)


print(f"{misc.model_size(model)} Parameters")
train_contrastive(model, test_dataloader, train_dataloader, Config, convex=False, start_epoch=14, views=2)

21425536 Parameters


  0%|          | 0/21 [00:04<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
from training.contrastive_training import evaluate_contrastive

model.mask_ratio = 0.25
same_song_contrastive_loss = evaluate_contrastive(model, test_dataloader, Config, test_masked=False)
print(same_song_contrastive_loss)

In [ ]:
from training.contrastive_training import evaluate_contrastive

model.mask_ratio = 0.0
same_song_contrastive_loss = evaluate_contrastive(model, test_dataloader, Config, test_masked=False)
print(same_song_contrastive_loss)

In [ ]:
from training.autoencoding_training import train_autoencode
train_autoencode(model, test_dataloader, train_dataloader, Config, show_graph=False)